# 競馬予想アプリ - 当日予測 v2

**最終更新: 2026-01-17 10:45（Playwright + nest_asyncio対応版）**

---

**Option B: 事前計算済み特徴量を使用**

## アーキテクチャ

```
【事前準備（ローカルPC - 週1回）】
1. モデル学習 → Supabase Storage にアップロード
2. 特徴量計算 → Supabase DB (horse_features) にアップロード

【当日予測（このノートブック - スマホからOK）】
1. Supabase Storage からモデルをダウンロード
2. netkeiba から当日の出馬表を取得
3. Supabase DB から事前計算済み特徴量を取得
4. レース固有の特徴量を計算
5. 予測実行・結果表示
```

## 使用方法
1. 左サイドバーの🔑からシークレットに以下を設定：
   - `SUPABASE_URL`
   - `SUPABASE_KEY`
2. セルを順番に実行

## 1. 環境セットアップ

In [ ]:
# 必要なパッケージをインストール
!pip install -q supabase requests beautifulsoup4 lxml pandas numpy lightgbm scikit-learn

In [ ]:
# Supabase認証情報の設定
try:
    from google.colab import userdata
    SUPABASE_URL = userdata.get('SUPABASE_URL')
    SUPABASE_KEY = userdata.get('SUPABASE_KEY')
    print("✓ Colabシークレットから認証情報を取得しました")
except:
    # 直接入力（テスト用）
    SUPABASE_URL = "https://your-project.supabase.co"  # @param {type:"string"}
    SUPABASE_KEY = "your-service-role-key"  # @param {type:"string"}
    print("⚠ 手動入力の認証情報を使用します")

# 接続テスト
from supabase import create_client
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
print(f"✓ Supabase接続: {SUPABASE_URL[:40]}...")

## 2. モデルのダウンロード

In [ ]:
import pickle
import io
import pandas as pd
import numpy as np

BUCKET_NAME = "models"

def list_available_models():
    """利用可能なモデル一覧を取得"""
    try:
        files = supabase.storage.from_(BUCKET_NAME).list()
        models = [f["name"] for f in files if f["name"].endswith(".pkl")]
        print(f"利用可能なモデル: {models}")
        return models
    except Exception as e:
        print(f"エラー: {e}")
        return []

def download_model(version: str = "v1"):
    """モデルをダウンロード"""
    filename = f"model_{version}.pkl"
    print(f"モデルをダウンロード中: {filename}")

    try:
        response = supabase.storage.from_(BUCKET_NAME).download(filename)
        buffer = io.BytesIO(response)
        model_data = pickle.load(buffer)
        print(f"✓ モデルをダウンロードしました")
        print(f"  バージョン: {model_data.get('model_version', 'unknown')}")
        print(f"  特徴量数: {len(model_data.get('feature_columns', []))}")
        return model_data
    except Exception as e:
        print(f"✗ ダウンロード失敗: {e}")
        return None

# 利用可能なモデル一覧を表示
list_available_models()

In [ ]:
# モデルをダウンロード
MODEL_VERSION = "v1"  # @param {type:"string"}

model_data = download_model(MODEL_VERSION)

if model_data:
    model = model_data["model"]
    scaler = model_data["scaler"]
    calibrator = model_data.get("calibrator")
    feature_columns = model_data["feature_columns"]
    print(f"\n✓ モデル準備完了（{len(feature_columns)}特徴量）")
else:
    print("\n✗ モデルのダウンロードに失敗しました")
    print("ローカルFastAPIでモデルを学習・アップロードしてください")

## 3. 事前計算済み特徴量の確認

In [ ]:
# 特徴量テーブルの件数を確認
try:
    horse_result = supabase.table("horse_features").select("id", count="exact").execute()
    jockey_result = supabase.table("jockey_features").select("id", count="exact").execute()

    print(f"✓ horse_features: {horse_result.count}件")
    print(f"✓ jockey_features: {jockey_result.count}件")

    if horse_result.count == 0:
        print("\n⚠ 特徴量が同期されていません！")
        print("ローカルFastAPIで POST /api/v1/model/sync-features を実行してください")
except Exception as e:
    print(f"✗ テーブル確認エラー: {e}")
    print("Supabaseでマイグレーション（003_horse_features.sql）を実行してください")

## 4. 当日レースデータの取得

In [ ]:
import requests
import time
import re
from datetime import date, datetime, timedelta
from bs4 import BeautifulSoup
from typing import Optional, List, Dict

# ブラウザ自動化設定（Colab用）
USE_BROWSER = True  # @param {type:"boolean"} Playwrightでオッズを取得

# Playwright（ブラウザ自動化）のセットアップ
playwright_instance = None
playwright_browser = None
playwright_context = None

if USE_BROWSER:
    print("Playwright環境をセットアップ中...")
    
    # nest_asyncioでColabのasyncio環境と共存させる
    !pip install -q playwright nest_asyncio
    !playwright install chromium
    
    import nest_asyncio
    nest_asyncio.apply()
    
    from playwright.sync_api import sync_playwright
    
    def get_browser_page():
        """Playwrightのブラウザページを取得"""
        global playwright_instance, playwright_browser, playwright_context
        
        if playwright_browser is None:
            playwright_instance = sync_playwright().start()
            playwright_browser = playwright_instance.chromium.launch(
                headless=True,
                args=[
                    '--no-sandbox',
                    '--disable-dev-shm-usage',
                    '--disable-gpu',
                ]
            )
            playwright_context = playwright_browser.new_context(
                user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
            )
        
        return playwright_context.new_page()
    
    def close_browser():
        """ブラウザを終了"""
        global playwright_instance, playwright_browser, playwright_context
        if playwright_context:
            playwright_context.close()
            playwright_context = None
        if playwright_browser:
            playwright_browser.close()
            playwright_browser = None
        if playwright_instance:
            playwright_instance.stop()
            playwright_instance = None
    
    # 動作確認
    try:
        test_page = get_browser_page()
        test_page.goto("https://www.google.com", timeout=30000)
        title = test_page.title()
        test_page.close()
        print(f"✓ Playwright準備完了（テストページ: {title}）")
    except Exception as e:
        print(f"⚠ Playwright初期化エラー: {e}")
        print("→ USE_BROWSER = False に設定してAPIフォールバックを使用します")
        USE_BROWSER = False
        close_browser()

# スクレイピング設定
SCRAPE_INTERVAL = 1.5
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
}

JRA_COURSE_CODES = {"01", "02", "03", "04", "05", "06", "07", "08", "09", "10"}
COURSE_NAMES = {
    "01": "札幌", "02": "函館", "03": "福島", "04": "新潟",
    "05": "東京", "06": "中山", "07": "中京", "08": "京都",
    "09": "阪神", "10": "小倉",
}

session = requests.Session()
session.headers.update(HEADERS)
last_request_time = 0

def fetch_html(url: str) -> str:
    global last_request_time
    elapsed = time.time() - last_request_time
    if elapsed < SCRAPE_INTERVAL:
        time.sleep(SCRAPE_INTERVAL - elapsed)

    response = session.get(url, timeout=30)
    last_request_time = time.time()

    if "EUC-JP" in response.text[:500] or "euc-jp" in response.text[:500].lower():
        response.encoding = "euc-jp"
    else:
        response.encoding = response.apparent_encoding or "utf-8"

    return response.text

def is_jra_race(race_id: str) -> bool:
    if len(race_id) >= 6:
        return race_id[4:6] in JRA_COURSE_CODES
    return False

print("✓ スクレイピング関数を定義しました")

In [ ]:
def scrape_race_list(target_date: date, jra_only: bool = True) -> List[Dict]:
    """指定日のレース一覧を取得（race_list_sub.htmlを使用）"""
    date_str = target_date.strftime("%Y%m%d")
    url = f"https://race.netkeiba.com/top/race_list_sub.html?kaisai_date={date_str}"

    html = fetch_html(url)
    soup = BeautifulSoup(html, "lxml")

    races = []
    seen_ids = set()

    for link in soup.find_all("a", href=True):
        href = link.get("href", "")
        match = re.search(r"race_id=(\d{12})", href)
        if match:
            race_id = match.group(1)
            if race_id not in seen_ids:
                if jra_only and not is_jra_race(race_id):
                    continue
                seen_ids.add(race_id)
                races.append({
                    "race_id": race_id,
                    "date": target_date.isoformat(),
                    "race_name": link.get_text(strip=True),
                })

    races.sort(key=lambda x: x["race_id"])
    return races


def get_odds_from_api(race_id: str) -> Dict[int, Dict]:
    """APIからオッズと人気を取得（従来方式）"""
    url = f"https://race.netkeiba.com/api/api_get_jra_odds.html?race_id={race_id}&type=1"
    try:
        response = session.get(url, timeout=10)
        data = response.json()
        
        if data.get("status") != "result":
            return {}
        
        odds_data = data.get("data", {}).get("odds", {}).get("1", {})
        
        result = {}
        for umaban, values in odds_data.items():
            horse_num = int(umaban)
            result[horse_num] = {
                "odds": float(values[0]) if values[0] else None,
                "popularity": int(values[2]) if values[2] else None,
            }
        return result
    except Exception as e:
        return {}


def get_odds_with_playwright(race_id: str, page=None) -> Dict[int, Dict]:
    """Playwrightでオッズを取得（JavaScript実行）"""
    close_page = False
    if page is None:
        page = get_browser_page()
        close_page = True
    
    try:
        url = f"https://race.netkeiba.com/odds/index.html?race_id={race_id}&type=b1"
        page.goto(url, timeout=30000)
        
        # オッズが読み込まれるまで待機（最大10秒）
        try:
            page.wait_for_function(
                "document.querySelector('#odds-1_01')?.textContent !== '---.-'",
                timeout=10000
            )
        except:
            pass  # タイムアウトしても続行
        
        time.sleep(0.5)  # 追加の安定待機
        
        result = {}
        # 馬番1〜18まで取得を試みる
        for i in range(1, 19):
            try:
                umaban = f"{i:02d}"
                odds_elem = page.query_selector(f"#odds-1_{umaban}")
                if odds_elem:
                    odds_text = odds_elem.text_content()
                    
                    if odds_text and odds_text != "---.-":
                        # 人気も取得を試みる
                        try:
                            row = odds_elem.evaluate("el => el.closest('tr')")
                            pop_elem = page.query_selector(f"tr:has(#odds-1_{umaban}) td.Popular span.Num")
                            popularity = int(pop_elem.text_content()) if pop_elem and pop_elem.text_content().isdigit() else None
                        except:
                            popularity = None
                        
                        result[i] = {
                            "odds": float(odds_text),
                            "popularity": popularity,
                        }
            except:
                continue
        
        return result
    except Exception as e:
        print(f"Playwright オッズ取得エラー ({race_id}): {e}")
        return {}
    finally:
        if close_page:
            page.close()


def scrape_shutuba(race_id: str, page=None) -> Dict:
    """出馬表を取得（レース前のデータ）"""
    url = f"https://race.netkeiba.com/race/shutuba.html?race_id={race_id}"
    
    response = session.get(url, timeout=30)
    if "EUC-JP" in response.text[:500] or "euc-jp" in response.text[:500].lower():
        response.encoding = "euc-jp"
    
    soup = BeautifulSoup(response.text, "lxml")

    race_data = {
        "race_id": race_id,
        "course": COURSE_NAMES.get(race_id[4:6], ""),
        "race_number": int(race_id[10:12]) if len(race_id) >= 12 else 0,
    }

    # レース名
    title_elem = soup.select_one(".RaceName")
    if title_elem:
        race_data["race_name"] = title_elem.get_text(strip=True)

    # 距離・馬場
    race_data_elem = soup.select_one(".RaceData01")
    if race_data_elem:
        text = race_data_elem.get_text()
        distance_match = re.search(r"(\d+)m", text)
        if distance_match:
            race_data["distance"] = int(distance_match.group(1))
        if "芝" in text:
            race_data["track_type"] = "芝"
        elif "ダート" in text or "ダ" in text:
            race_data["track_type"] = "ダート"

    # オッズを取得（Playwright優先、フォールバックでAPI）
    odds_data = {}
    if USE_BROWSER:
        odds_data = get_odds_with_playwright(race_id, page)
    
    if not odds_data:
        odds_data = get_odds_from_api(race_id)
    
    if not odds_data:
        print(f"⚠ オッズ取得失敗: {race_id} - デフォルト値10.0を使用")

    # 出走馬
    entries = []
    for row in soup.select("tr.HorseList"):
        entry = parse_shutuba_row(row)
        if entry:
            horse_num = entry.get("horse_number")
            if horse_num and horse_num in odds_data:
                entry["odds"] = odds_data[horse_num]["odds"]
                entry["popularity"] = odds_data[horse_num]["popularity"]
            entries.append(entry)

    race_data["entries"] = entries
    race_data["field_size"] = len(entries)
    return race_data


def parse_shutuba_row(row) -> Optional[Dict]:
    """出馬表の行をパース"""
    entry = {}
    
    tds = row.find_all("td")
    
    if len(tds) > 0:
        try:
            entry["frame_number"] = int(tds[0].get_text(strip=True))
        except:
            pass

    if len(tds) > 1:
        try:
            entry["horse_number"] = int(tds[1].get_text(strip=True))
        except:
            return None

    horse_link = row.select_one("span.HorseName a")
    if horse_link:
        href = horse_link.get("href", "")
        match = re.search(r"/horse/(\d+)", href)
        if match:
            entry["horse_id"] = match.group(1)
        entry["horse_name"] = horse_link.get_text(strip=True)

    jockey_td = row.select_one("td.Jockey a")
    if jockey_td:
        href = jockey_td.get("href", "")
        match = re.search(r"/jockey/(?:result/recent/)?(\d+)", href)
        if match:
            entry["jockey_id"] = match.group(1)
        entry["jockey_name"] = jockey_td.get_text(strip=True)

    if len(tds) > 5:
        try:
            entry["weight"] = float(tds[5].get_text(strip=True))
        except:
            pass

    return entry if entry.get("horse_number") else None


print("✓ 出馬表スクレイピング関数を定義しました（Playwright対応）")

In [ ]:
# 対象日を設定
target_date = date.today()  # 今日
# target_date = date(2025, 1, 12)  # 特定の日付

print(f"対象日: {target_date}")

# レース一覧を取得
races = scrape_race_list(target_date, jra_only=True)
print(f"\n{len(races)}件のJRAレースが見つかりました")

for race in races[:10]:
    print(f"  - {race['race_id']}: {race['race_name']}")

## 5. 特徴量の生成（事前計算済み特徴量を使用）

In [ ]:
def get_horse_features_batch(horse_ids: List[str]) -> Dict[str, Dict]:
    """複数馬の事前計算済み特徴量を一括取得"""
    if not horse_ids:
        return {}

    try:
        result = supabase.table("horse_features").select("*").in_("horse_id", horse_ids).execute()
        return {r["horse_id"]: r for r in result.data}
    except Exception as e:
        print(f"特徴量取得エラー: {e}")
        return {}

def get_jockey_features_batch(jockey_ids: List[str]) -> Dict[str, Dict]:
    """複数騎手の事前計算済み特徴量を一括取得"""
    if not jockey_ids:
        return {}

    try:
        result = supabase.table("jockey_features").select("*").in_("jockey_id", jockey_ids).execute()
        return {r["jockey_id"]: r for r in result.data}
    except Exception as e:
        print(f"騎手特徴量取得エラー: {e}")
        return {}

print("✓ 特徴量取得関数を定義しました")

In [ ]:
def create_features_v2(race_data: Dict, horse_features: Dict, jockey_features: Dict) -> pd.DataFrame:
    """
    レースデータと事前計算済み特徴量から予測用特徴量を生成

    事前計算済み特徴量 + レース固有特徴量 = 予測用特徴量
    """
    entries = race_data.get("entries", [])
    if not entries:
        return pd.DataFrame()

    features_list = []
    distance = race_data.get("distance", 1600)
    track_type = race_data.get("track_type", "芝")
    field_size = race_data.get("field_size", len(entries))

    # 季節特徴量
    race_date = date.today()
    month = race_date.month
    season = (month % 12) // 3 + 1  # 1:冬, 2:春, 3:夏, 4:秋

    for entry in entries:
        horse_id = entry.get("horse_id")
        jockey_id = entry.get("jockey_id")

        # 基本特徴量（レースから取得）
        features = {
            "horse_number": entry.get("horse_number"),
            "umaban": entry.get("horse_number"),
            "frame_number": entry.get("frame_number", 0),
            "weight": entry.get("weight", 55),
            "distance": distance,
            "track_type": 1 if track_type == "芝" else 0,
            "field_size": field_size,
            "race_number": race_data.get("race_number", 1),
            "odds": entry.get("odds", 10),
            "log_odds": np.log(entry.get("odds", 10) + 1),
            "popularity": entry.get("popularity", 10),
            # 季節
            "season": season,
            "month": month,
            "is_spring": 1 if 3 <= month <= 5 else 0,
            "is_summer": 1 if 6 <= month <= 8 else 0,
            "is_autumn": 1 if 9 <= month <= 11 else 0,
            "is_winter": 1 if month <= 2 or month == 12 else 0,
            "month_sin": np.sin(2 * np.pi * month / 12),
            "month_cos": np.cos(2 * np.pi * month / 12),
        }

        # 馬の事前計算済み特徴量を追加
        if horse_id and horse_id in horse_features:
            hf = horse_features[horse_id]
            for key in [
                "horse_age", "horse_sex",
                "avg_rank_last3", "avg_rank_last5", "avg_rank_last10", "avg_rank_all",
                "prize_3races", "prize_5races", "prize_10races",
                "win_rate", "place_rate", "show_rate", "best_rank", "total_runs",
                "days_since_last", "last_result",
                "avg_last3f", "best_last3f",
                "running_style", "avg_first_corner", "avg_last_corner",
                "position_up_avg", "escape_rate", "front_rate", "stalker_rate", "closer_rate",
                "avg_pace_first", "avg_pace_second", "avg_pace_diff", "pace_consistency",
                "high_pop_win_rate", "high_pop_show_rate", "high_pop_runs",
                "mid_pop_win_rate", "mid_pop_show_rate", "mid_pop_runs",
                "low_pop_win_rate", "low_pop_show_rate", "low_pop_runs",
                "avg_odds_when_win",
            ]:
                features[key] = hf.get(key)

            # 距離適性（レース距離に応じて選択）
            if distance <= 1400:
                features["distance_win_rate"] = hf.get("short_win_rate")
                features["distance_runs"] = hf.get("short_runs")
            elif distance <= 1800:
                features["distance_win_rate"] = hf.get("mile_win_rate")
                features["distance_runs"] = hf.get("mile_runs")
            elif distance <= 2200:
                features["distance_win_rate"] = hf.get("middle_win_rate")
                features["distance_runs"] = hf.get("middle_runs")
            else:
                features["distance_win_rate"] = hf.get("long_win_rate")
                features["distance_runs"] = hf.get("long_runs")

            # 芝/ダート適性
            if track_type == "芝":
                features["track_win_rate"] = hf.get("turf_win_rate")
                features["track_runs"] = hf.get("turf_runs")
            else:
                features["track_win_rate"] = hf.get("dirt_win_rate")
                features["track_runs"] = hf.get("dirt_runs")

        # 騎手の事前計算済み特徴量を追加
        if jockey_id and jockey_id in jockey_features:
            jf = jockey_features[jockey_id]
            features["jockey_win_rate"] = jf.get("win_rate")
            features["jockey_place_rate"] = jf.get("place_rate")
            features["jockey_show_rate"] = jf.get("show_rate")
            features["jockey_year_wins"] = jf.get("year_wins")
            features["jockey_year_rides"] = jf.get("year_rides")

        features_list.append(features)

    df = pd.DataFrame(features_list)

    # 足りない特徴量を0で埋める
    for col in feature_columns:
        if col not in df.columns:
            df[col] = 0

    return df

print("✓ 特徴量生成関数を定義しました")

## 6. 予測の実行

In [ ]:
def predict_race_v2(race_data: Dict, horse_features: Dict, jockey_features: Dict) -> Dict:
    """レースの予測を実行（v2: 事前計算済み特徴量使用）"""
    df = create_features_v2(race_data, horse_features, jockey_features)

    if df.empty:
        return None

    # 特徴量を選択
    X = df[feature_columns].copy()
    X = X.fillna(0)

    # スケーリング
    X_scaled = pd.DataFrame(
        scaler.transform(X),
        columns=X.columns,
        index=X.index,
    )

    # 予測（LightGBM二値分類は直接確率を出力）
    probs = model.predict(X_scaled)

    # 確率の正規化（レース内で合計が1になるように）
    # ※ソフトマックスは使わない - LightGBMは既に確率を出力しているため
    total = probs.sum()
    if total > 0:
        probabilities = probs / total
    else:
        probabilities = probs

    # 結果を整形
    results = []
    entries = race_data.get("entries", [])

    for i, entry in enumerate(entries):
        odds = entry.get("odds") or 10
        prob = float(probabilities[i])
        ev = prob * odds

        results.append({
            "horse_number": entry.get("horse_number"),
            "horse_name": entry.get("horse_name", "不明"),
            "jockey_name": entry.get("jockey_name", ""),
            "score": float(probs[i]),  # 元の確率を保持
            "probability": prob,
            "odds": odds,
            "expected_value": ev,
            "popularity": entry.get("popularity"),
        })

    # スコア順にソート
    results.sort(key=lambda x: x["score"], reverse=True)

    # 順位付け
    for i, r in enumerate(results):
        r["pred_rank"] = i + 1

    return {
        "race_id": race_data["race_id"],
        "race_name": race_data.get("race_name", ""),
        "course": race_data.get("course", ""),
        "race_number": race_data.get("race_number"),
        "distance": race_data.get("distance"),
        "track_type": race_data.get("track_type"),
        "predictions": results,
    }

print("✓ 予測関数を定義しました（バックエンドと同じロジック）")

In [ ]:
from tqdm.notebook import tqdm

# Playwrightページを1つ起動して使い回す（高速化）
page = None
if USE_BROWSER:
    print("ブラウザページ起動中...")
    page = get_browser_page()
    print("✓ ブラウザページ起動完了")

# 全レースの予測を実行
all_predictions = []

try:
    for race_info in tqdm(races, desc="予測実行中"):
        try:
            # 出馬表を取得（pageを渡す）
            race_data = scrape_shutuba(race_info["race_id"], page=page)

            # 馬IDと騎手IDを収集
            horse_ids = [e.get("horse_id") for e in race_data.get("entries", []) if e.get("horse_id")]
            jockey_ids = [e.get("jockey_id") for e in race_data.get("entries", []) if e.get("jockey_id")]

            # 事前計算済み特徴量を取得
            horse_features = get_horse_features_batch(horse_ids)
            jockey_features = get_jockey_features_batch(jockey_ids)

            # 予測実行
            prediction = predict_race_v2(race_data, horse_features, jockey_features)

            if prediction:
                all_predictions.append(prediction)

        except Exception as e:
            print(f"エラー: {race_info['race_id']} - {e}")

finally:
    # ページを閉じる
    if page:
        page.close()
        print("✓ ブラウザページ終了")

print(f"\n✓ {len(all_predictions)}レースの予測が完了しました")

## 7. 予測結果の表示

In [ ]:
def display_prediction(pred: Dict, show_all: bool = False):
    """予測結果を表示"""
    print(f"\n{'='*70}")
    print(f"{pred['course']} {pred.get('race_number', '')}R {pred['race_name']}")
    print(f"{pred.get('track_type', '')} {pred.get('distance', '')}m")
    print(f"{'='*70}")
    print(f"{'予想':>4} {'馬番':>4} {'馬名':<14} {'騎手':<8} {'勝率':>6} {'オッズ':>6} {'期待値':>6} {'人気':>4}")
    print("-" * 70)

    display_list = pred["predictions"] if show_all else pred["predictions"][:5]

    for p in display_list:
        prob_pct = (p["probability"] or 0) * 100
        odds = p.get('odds')
        odds_str = f"{odds:.1f}" if odds else "-"
        ev = p.get("expected_value") or 0
        pop = p.get("popularity") or "-"

        # 期待値マーク
        if ev >= 2.0:
            ev_mark = "★★"
        elif ev >= 1.5:
            ev_mark = "★"
        elif ev >= 1.0:
            ev_mark = "◎"
        else:
            ev_mark = ""

        print(f"{p['pred_rank']:>4} {p['horse_number']:>4} {p['horse_name']:<14} {p.get('jockey_name', '') or '':<8} {prob_pct:>5.1f}% {odds_str:>6} {ev:>5.2f}{ev_mark:<2} {pop:>4}")

# 各レースの予測を表示
for pred in all_predictions:
    display_prediction(pred)

## 8. 期待値が高い馬の一覧

In [ ]:
# 期待値が高い馬を抽出（シミュレーションと同じデフォルト値）
EV_THRESHOLD = 1.0  # @param {type:"number"} 期待値下限
MAX_EV = 2.0        # @param {type:"number"} 期待値上限（高すぎる穴馬を除外）
MIN_PROB = 0.05     # @param {type:"number"} 最低確率5%（シミュレーションと同じ）
TOP_N_PER_RACE = 3  # @param {type:"number"} レースごとの最大推奨数

high_ev_horses = []

for pred in all_predictions:
    race_candidates = []
    
    for p in pred["predictions"]:
        odds = p.get("odds") or 0
        prob = p.get("probability") or 0
        ev = p.get("expected_value") or 0
        
        # フィルター条件（シミュレーションと同じ）
        if ev >= EV_THRESHOLD and ev <= MAX_EV and prob >= MIN_PROB:
            race_candidates.append({
                "course": pred['course'],
                "race_number": pred.get('race_number', 0),
                "race": f"{pred['course']}{pred.get('race_number', '')}R",
                "race_name": pred["race_name"],
                **p
            })
    
    # レースごとに期待値上位N件のみ
    race_candidates.sort(key=lambda x: x["expected_value"], reverse=True)
    high_ev_horses.extend(race_candidates[:TOP_N_PER_RACE])

# レース順にソート
high_ev_horses.sort(key=lambda x: (x["course"], x["race_number"], -x["expected_value"]))

print(f"\n推奨馬（EV {EV_THRESHOLD}〜{MAX_EV}、確率{MIN_PROB*100:.0f}%以上、各レース上位{TOP_N_PER_RACE}件）: {len(high_ev_horses)}頭")
print("=" * 80)
print(f"{'レース':<12} {'馬番':>4} {'馬名':<14} {'勝率':>6} {'オッズ':>6} {'期待値':>6}")
print("-" * 80)

for h in high_ev_horses:
    prob_pct = h["probability"] * 100
    print(f"{h['race']:<12} {h['horse_number']:>4} {h['horse_name']:<14} {prob_pct:>5.1f}% {h['odds']:>6.1f} {h['expected_value']:>6.2f}")

## 9. 個別レース予測（オッズ更新対応）

特定のレースを指定して、最新オッズで予測を再実行できます。

In [ ]:
# レース一覧を表示
print("=== 本日のレース一覧 ===\n")
for i, race in enumerate(races):
    course = COURSE_NAMES.get(race["race_id"][4:6], "")
    race_num = int(race["race_id"][10:12])
    print(f"{i+1:2d}. {course}{race_num}R: {race['race_name']} (ID: {race['race_id']})")

In [ ]:
# 個別レース予測
# 上の一覧から番号を指定するか、race_idを直接入力
RACE_INDEX = 1  # @param {type:"integer"} 一覧の番号（1始まり）
# または直接race_idを指定（空欄の場合は上の番号を使用）
RACE_ID = ""  # @param {type:"string"}

# レースを特定
if RACE_ID:
    target_race_id = RACE_ID
else:
    if 1 <= RACE_INDEX <= len(races):
        target_race_id = races[RACE_INDEX - 1]["race_id"]
    else:
        print(f"エラー: 番号は1〜{len(races)}の範囲で指定してください")
        target_race_id = None

if target_race_id:
    print(f"予測対象: {target_race_id}")
    print(f"現在時刻: {datetime.now().strftime('%H:%M:%S')}")
    print("-" * 50)
    
    # 出馬表を取得（最新オッズ）
    race_data = scrape_shutuba(target_race_id)
    
    # 特徴量取得
    horse_ids = [e.get("horse_id") for e in race_data.get("entries", []) if e.get("horse_id")]
    jockey_ids = [e.get("jockey_id") for e in race_data.get("entries", []) if e.get("jockey_id")]
    hf = get_horse_features_batch(horse_ids)
    jf = get_jockey_features_batch(jockey_ids)
    
    # 予測実行
    pred = predict_race_v2(race_data, hf, jf)
    
    if pred:
        # 結果表示
        print(f"\n{'='*70}")
        print(f"{pred['course']} {pred.get('race_number', '')}R {pred['race_name']}")
        print(f"{pred.get('track_type', '')} {pred.get('distance', '')}m")
        print(f"{'='*70}")
        print(f"{'予想':>4} {'馬番':>4} {'馬名':<14} {'勝率':>6} {'オッズ':>6} {'期待値':>6} {'人気':>4}")
        print("-" * 70)
        
        for p in pred["predictions"]:
            prob_pct = (p["probability"] or 0) * 100
            odds = p.get('odds')
            odds_str = f"{odds:.1f}" if odds else "-"
            ev = p.get("expected_value") or 0
            pop = p.get("popularity") or "-"
            
            # 推奨マーク（EV 1.0〜2.0、確率5%以上）
            if EV_THRESHOLD <= ev <= MAX_EV and (p.get("probability") or 0) >= MIN_PROB:
                mark = "★"
            else:
                mark = ""
            
            print(f"{p['pred_rank']:>4} {p['horse_number']:>4} {p['horse_name']:<14} {prob_pct:>5.1f}% {odds_str:>6} {ev:>5.2f}{mark:<2} {pop:>4}")
        
        # 推奨馬のみ抽出（シミュレーションと同じ条件）
        recommended = [p for p in pred["predictions"] 
                       if EV_THRESHOLD <= (p.get("expected_value") or 0) <= MAX_EV 
                       and (p.get("probability") or 0) >= MIN_PROB]
        if recommended:
            print(f"\n【推奨馬（EV {EV_THRESHOLD}〜{MAX_EV}、確率{MIN_PROB*100:.0f}%以上）】")
            for p in sorted(recommended, key=lambda x: x["expected_value"], reverse=True)[:TOP_N_PER_RACE]:
                print(f"  馬番{p['horse_number']:2d} {p['horse_name']}: EV={p['expected_value']:.2f}, オッズ={p['odds']:.1f}")
    else:
        print("予測に失敗しました")

---

## 使い方まとめ

### 事前準備（週1回、ローカルPCで実行）

1. **モデル学習**
   ```
   POST /api/v1/model/retrain
   ```

2. **モデルをSupabase Storageにアップロード**
   ```
   POST /api/v1/model/storage/upload
   ```

3. **特徴量をSupabase DBに同期**
   ```
   POST /api/v1/model/sync-features
   ```

### 当日予測（このノートブック）

1. セル1〜6を実行（環境セットアップ〜予測実行）
2. 「期待値が高い馬の一覧」で買い目を確認

### スマホからの使い方

1. Google Colabアプリをインストール
2. このノートブックを開く
3. 「すべて実行」で予測完了